<a href="https://colab.research.google.com/github/woncoh1/END/blob/main/Assignment_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instructions  
Use this as a [reference](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb). 

Change this code in such a way that:

- it has **3** LSTM layers &
- it has used a **for loop** to do so in the **forward** function
  - see **2. Build the Model**, *1) define model structure*
- the dropout value used is **0.2**  
  - see **2. Build the Model**, *2) define model constants*
- trained on the text that is **reversed** (for example, "my name is Rohan"
 becomes "Rohan is name my")
  - see **1. Prepare Data**, *3) reverse training text data in-place*
  - also, `bidirectional=False`
  - lastly, slight modifications in the `RNN` class definition (`__init__` and `forward`)
- achieves **87% or more** accuracy  
  - After 7 epochs, validation accuracy = **87.93%** and testing accuracy = **86.58%**

- once done, share the **Github link** as well (after training on Google Colab, move the file to GitHub).  

# 1. Prepare Data

### Setup

In [ ]:
# 1) setup preprocessing

import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Split training, validation and testing data

In [ ]:
# 2) split training and testing data

from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [ ]:
# 3) split validation data from training data

import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

### Reverse text in training data

In [ ]:
# 4) reverse training text data in-place,
#    leaving validation and testing data unchanged

for train_datum in train_data.examples:
    vars(train_datum).get('text').reverse()

### Build vocabulary using pre-trained word embeddings

In [ ]:
# 5) build vocabulary using pre-trained word embeddings

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

### Build batches of split data 

In [ ]:
# 6) build batches

BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device) # send data to GPU (if available)

#2. Build the Model

### Define and instantiate model

In [ ]:
# 1) define model (neural network) structure

import torch.nn as nn

class RNN(nn.Module):

    #parts list for building blocks
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 n_layers, bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, 
                                      padding_idx = pad_idx)
        
        #the 3 lstm layers
        self.rnns = nn.ModuleList([nn.LSTM(embedding_dim, hidden_dim, 
                                           bidirectional=bidirectional)])
        for _ in range(n_layers - 1):
            self.rnns.append(nn.LSTM(hidden_dim, hidden_dim, 
                                     bidirectional=bidirectional))
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
    
    #step-by-step manual for assembling building blocks    
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        #stack multiple (3) LSTM layers with dropouts in between
        x = packed_embedded
        for rnn in self.rnns[:-1]:
            x, (_, _) = rnn(x) # cf. _, (x, _)
            x, length = nn.utils.rnn.pad_packed_sequence(x)
            x = self.dropout(x) 
            x = nn.utils.rnn.pack_padded_sequence(x, length)
        _, (hidden, _) = self.rnns[-1](x) # dropout only between LSTM layers

        #hidden = final layer's final hidden state = [1, batch size, hid dim]
            
        return self.fc(hidden.squeeze(0)) #softmax depends on the loss function

In [ ]:
# 2) define model constants (hyperparameters)

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 3          
BIDIRECTIONAL = False #changed from True to False
DROPOUT = 0.2         #changed from 0.5 to 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [ ]:
# 3) instantiate and check model

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

print(model)

RNN(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (rnns): ModuleList(
    (0): LSTM(100, 256)
    (1): LSTM(256, 256)
    (2): LSTM(256, 256)
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [ ]:
# 4) count number of parameters (weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,919,721 trainable parameters


### Copy pre-trained word embeddings into model's embedding layer

In [ ]:
# 5) inspect pre-trained word embeddings

pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [ ]:
# 6) copy pre-trained word embeddings from vocabulary to model

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1007, -1.4210, -0.1203,  ...,  0.6521,  0.6981,  0.5187],
        [-0.6459,  0.3102, -1.3546,  ...,  0.7732,  0.4767,  0.3603],
        [ 0.0668,  0.5928, -1.6730,  ...,  0.2781,  0.4265,  0.0204]])

In [ ]:
# 7) zero weights for <unk> and <pad> tokens

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1007, -1.4210, -0.1203,  ...,  0.6521,  0.6981,  0.5187],
        [-0.6459,  0.3102, -1.3546,  ...,  0.7732,  0.4767,  0.3603],
        [ 0.0668,  0.5928, -1.6730,  ...,  0.2781,  0.4265,  0.0204]])


#3. Train the Model

### Build loss and optimizer

In [ ]:
# 1) build loss function (Sigmoid + Binary Cross-Entropy Loss in one layer)

criterion = nn.BCEWithLogitsLoss()

In [ ]:
# 2) build optimizer (Adam)

import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-3) # learning rate adjustable

In [ ]:
# 3) send model and loss to GPU (if available), while
#    optimizer not sent to GPU (see https://github.com/pytorch/pytorch/issues/8741)

model = model.to(device)
criterion = criterion.to(device)

### Prepare for training



In [ ]:
# 4) define how to calculate accuracy for training, validation and testing

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, 
    i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
# 5) define how to train the model

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text

        text_lengths = text_lengths.cpu() #runtime type is GPU, while model expects CPU tensor

        predictions = model(text, text_lengths).squeeze(1) #cf. squeeze(0) if batch_first=True
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator) #average loss and accuracy per batch

In [ ]:
# 6) define how to evaluate the model

def evaluate(model, iterator, criterion): #no optimizer
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval() #no dropout, batch normalization, etc
    
    with torch.no_grad(): #no gradient calculation
    
        for batch in iterator:

            #no optimizer.zero_grad()

            text, text_lengths = batch.text
            
            text_lengths = text_lengths.cpu()

            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            #no loss.backward() & optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
# 7) define how to calculate time spent per epoch

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Carry out training

In [ ]:
# 8) data, model, loss, and optimizer all come together to actually train model

N_EPOCHS = 13

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt') #save best parameters for testing
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 32s
	Train Loss: 0.659 | Train Acc: 60.82%
	 Val. Loss: 0.651 |  Val. Acc: 61.46%
Epoch: 02 | Epoch Time: 0m 32s
	Train Loss: 0.694 | Train Acc: 51.89%
	 Val. Loss: 0.717 |  Val. Acc: 49.56%
Epoch: 03 | Epoch Time: 0m 32s
	Train Loss: 0.696 | Train Acc: 49.83%
	 Val. Loss: 0.693 |  Val. Acc: 50.79%
Epoch: 04 | Epoch Time: 0m 32s
	Train Loss: 0.694 | Train Acc: 49.79%
	 Val. Loss: 0.693 |  Val. Acc: 50.79%
Epoch: 05 | Epoch Time: 0m 32s
	Train Loss: 0.694 | Train Acc: 50.11%
	 Val. Loss: 0.695 |  Val. Acc: 50.79%
Epoch: 06 | Epoch Time: 0m 32s
	Train Loss: 0.693 | Train Acc: 50.18%
	 Val. Loss: 0.691 |  Val. Acc: 50.07%
Epoch: 07 | Epoch Time: 0m 32s
	Train Loss: 0.598 | Train Acc: 66.88%
	 Val. Loss: 0.419 |  Val. Acc: 82.39%
Epoch: 08 | Epoch Time: 0m 32s
	Train Loss: 0.324 | Train Acc: 87.74%
	 Val. Loss: 0.340 |  Val. Acc: 86.46%
Epoch: 09 | Epoch Time: 0m 32s
	Train Loss: 0.237 | Train Acc: 91.15%
	 Val. Loss: 0.319 |  Val. Acc: 86.71%
Epoch: 10 | Epoch T

### Test the trained model

In [ ]:
# 9) test model using testing dataset

model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.343 | Test Acc: 86.58%


#4. Test with User Input

In [ ]:
# 1) define how to test the model using arbitrary user input

import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
# 2) test the model using a negative sentence

predict_sentiment(model, "This film is terrible")

0.10009177029132843

In [ ]:
# 3) test the model using a positive sentence

predict_sentiment(model, "This film is great")

0.6731292009353638